In [10]:
import pandas as pd
#df = pd.read_csv('hkcensus2011/scripts/data/combined/census.csv')
# census-e.csv is superset of census.csv
# It incorporates natural language names besides the identifiers
df = pd.read_csv('hkcensus2011/scripts/data/combined/census-e.csv')

In [12]:
df.head()

,region,district,area,table,row,column,value,region_name,district_name,area_name,table_name,row_name,column_name
0,nt,p,p03,0,tab0_chinese,tab0_male,6884,New Territories,Tai Po,Chung Ting,Ethnicity,Chinese,Male
1,nt,p,p02,0,tab0_chinese,tab0_male,6497,New Territories,Tai Po,Tai Po Central,Ethnicity,Chinese,Male
2,nt,p,p01,0,tab0_chinese,tab0_male,7432,New Territories,Tai Po,Tai Po Hui,Ethnicity,Chinese,Male
3,nt,p,p07,0,tab0_chinese,tab0_male,6801,New Territories,Tai Po,Fu Ming Sun,Ethnicity,Chinese,Male
4,nt,p,p06,0,tab0_chinese,tab0_male,7912,New Territories,Tai Po,Yee Fu,Ethnicity,Chinese,Male


In [18]:
print(len(df))
area = df[df.area=='a01']
print(len(area))

171970
444


In [30]:
# Show the mapping of tables. 
columns = area[['table', 'table_name']]
columns.drop_duplicates()

,table,table_name
253,0,Ethnicity
6325,1,Usual Language
13741,2,Marital Status
21157,3,Educational Attainment
29809,4,Economic Activity Status
39697,5,Monthly Income from Main Employment (HK$)
55765,6,Household Composition
59061,7,Household Size
61533,8,Type of Housing
67713,9,Tenure of Accommodation


In [120]:
def identifier_to_cell(identifier):
    cell = identifier.split('_')[0]
    column = cell[0]
    row = int(cell[1:])
    return (column, row)

def series_to_median(df):
    #print('-----')
    #print(df)
    df['id_tuple'] = df['row'].apply(identifier_to_cell)
    df.sort('id_tuple')
    s = df['value'].sum()
    acc = 0.0 
    for i, row in df.iterrows():
        acc += row['value']
        if acc >= s / 2.0:
            # Return DataFrame rather than Series
            return pd.DataFrame([tuple(row.values)], columns=list(row.index), index=[i])

STRATEGY = [
    # nane, table, column, 
    ('Median Age Group Male', 12, 'l6_male', series_to_median),
    ('Median Age Group Female', 12, 'm6_female', series_to_median),
    ('Median Age Group Both', 12, 'n6_both', series_to_median),
    ('Median Monthly Domestic Household Income Active', 18, 'l118_active', series_to_median),
    ('Median Monthly Domestic Household Income All', 18, 'n118_households', series_to_median),
    ('Median Monthly Domestic Household Mortgage Payment and Loan Repayment (HK$)', 19, 'n143_with', series_to_median),
    ('Median Monthly Domestic Household Rent (HK$)', 20, 'n162_they', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Male', 5, 'c77_male', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Female', 5, 'd77_female', series_to_median),
    ('Median Monthly Income from Main Employment (HK$) Both', 5, 'e77_both', series_to_median),
    ('Median Household Size', 7, 'e129_domestic', series_to_median),
]

area_profile = []
for s in STRATEGY:
    #print(area[area.table==s[1]])
    filtered_data = area[(area.table==s[1])][(area.column==s[2])]
    re = s[3](filtered_data)
    #print(re[['area', 'area_name', 'row', 'row_name', 'value']])
    stat_str = re.iloc[0]['row_name']
    area_str = re.iloc[0]['area_name']
    area_profile.append(stat_str)
print(area_profile)

#df.sort('id_tuple')
#sorted(list(area[area.table==12][area.column=='l6_male'].row), key=identifier_to_cell)

['40 - 44', '40 - 44', '40 - 44', '30,000 - 39,999', '30,000 - 39,999', '10,000 - 14,999', '8,000 - 9,999', '15,000 - 19,999', '8,000 - 9,999', '15,000 - 19,999', '2']


/Users/hupili/Desktop/initium/hk_median/venv/lib/python3.4/site-packages/pandas/core/frame.py:1819: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [116]:
area[area.table==7]

,region,district,area,table,row,column,value,region_name,district_name,area_name,table_name,row_name,column_name
59061,hk,a,a01,7,a130_1,e129_domestic,2239,Hong Kong Island,Central and Western,Chung Wan,Household Size,1,Households
59473,hk,a,a01,7,a131_2,e129_domestic,1365,Hong Kong Island,Central and Western,Chung Wan,Household Size,2,Households
59885,hk,a,a01,7,a132_3,e129_domestic,1081,Hong Kong Island,Central and Western,Chung Wan,Household Size,3,Households
60297,hk,a,a01,7,a133_4,e129_domestic,648,Hong Kong Island,Central and Western,Chung Wan,Household Size,4,Households
60709,hk,a,a01,7,a134_5,e129_domestic,302,Hong Kong Island,Central and Western,Chung Wan,Household Size,5,Households
61121,hk,a,a01,7,a135_6,e129_domestic,225,Hong Kong Island,Central and Western,Chung Wan,Household Size,6+,Households
